In [18]:
from snowflake.snowpark.session import Session
from snowflake.snowpark import functions as F
from snowflake.snowpark.types import *

import pandas as pd



#Snowflake connection info is saved in config.py
from config import snowflake_conn_prop


# lets import some tranformations functions
from snowflake.snowpark.functions import udf, col, lit, translate, is_null, iff

In [19]:
from snowflake.snowpark import version
print(version.VERSION)

(0, 5, 0, None)


In [20]:
from snowflake.snowpark import version
print(version.VERSION)
#session.close()
session = Session.builder.configs(snowflake_conn_prop).create()
session.sql("use role accountadmin").collect()
session.sql("create database if not exists  {}".format(snowflake_conn_prop['database'])).collect()
session.sql("use database {}".format(snowflake_conn_prop['database'])).collect()
session.sql("create schema if not exists {}".format(snowflake_conn_prop['schema'])).collect()
session.sql("use schema {}".format(snowflake_conn_prop['schema'])).collect()
session.sql("create or replace warehouse {} with \
                WAREHOUSE_SIZE = XSMALL \
                AUTO_SUSPEND = 120 \
                AUTO_RESUME = TRUE".format(snowflake_conn_prop['warehouse'])).collect()
session.sql("use warehouse {}".format(snowflake_conn_prop['warehouse']))
print(session.sql('select current_warehouse(), current_database(), current_schema()').collect())

(0, 5, 0, None)
[Row(CURRENT_WAREHOUSE()='COMPUTE_WH', CURRENT_DATABASE()='DEV', CURRENT_SCHEMA()='LANDING')]


In [21]:
filename = "raw_telco_data.parquet"
stagename = "rawdata"
rawtable = "RAW_PARQUET_DATA"
view = "RAW_PARQUET_DATA_VW"

In [22]:
session.sql(f"create or replace stage {stagename} DIRECTORY = (ENABLE = TRUE);").collect()
session.file.put(filename,stagename)

[PutResult(source='raw_telco_data.parquet', target='raw_telco_data.parquet', source_size=3037540, target_size=3037552, source_compression='PARQUET', target_compression='PARQUET', status='UPLOADED', message='')]

In [23]:
session.sql("CREATE OR REPLACE FILE FORMAT MY_PARQUET_FORMAT TYPE = PARQUET;").collect()

session.sql(f"CREATE OR REPLACE \
            TABLE {rawtable} (COL1 VARIANT);  ").collect()

[Row(status='Table RAW_PARQUET_DATA successfully created.')]

In [24]:
dfClear = session.table(rawtable).delete()

In [25]:
dfRaw = session.read.option("compression","snappy").parquet(f"@{stagename}/{filename}")
dfRaw.copy_into_table(rawtable,FORCE= True)

[Row(file='rawdata/raw_telco_data.parquet', status='LOADED', rows_parsed=100000, rows_loaded=100000, error_limit=1, errors_seen=0, first_error=None, first_error_line=None, first_error_character=None, first_error_column_name=None)]

In [26]:
session.sql('create or replace secure view RAW_PARQUET_DATA_VW as select COL1:"CUSTOMERID"::TEXT AS "CUSTOMERID", COL1:"GENDER"::TEXT AS "GENDER",COL1:"PARTNER"::BOOLEAN AS "PARTNER",COL1:"DEPENDENTS"::BOOLEAN AS "DEPENDENTS",COL1:"COUNTRY"::TEXT AS "COUNTRY",COL1:"CITY"::TEXT AS "CITY",COL1:"STATE"::TEXT AS "STATE",COL1:"LONGITUDE"::TEXT AS "LONGITUDE",COL1:"PHONE SERVICE"::TEXT AS "PHONE SERVICE",COL1:"MULTIPLE LINES"::TEXT AS "MULTIPLE LINES",COL1:"SENIOR CITIZEN"::BOOLEAN AS "SENIOR CITIZEN",COL1:"MONTHLY CHARGES"::REAL AS "MONTHLY CHARGES",COL1:"STREAMING MOVIES"::TEXT AS "STREAMING MOVIES",COL1:"PAYMENT METHOD"::TEXT AS "PAYMENT METHOD",COL1:"LAT LONG"::TEXT AS "LAT LONG",COL1:"TENURE MONTHS"::NUMBER(38, 0) AS "TENURE MONTHS",COL1:"COUNT"::NUMBER(38, 0) AS "COUNT",COL1:"PAPERLESS BILLING"::BOOLEAN AS "PAPERLESS BILLING",COL1:"INTERNET SERVICE"::TEXT AS "INTERNET SERVICE",COL1:"ONLINE BACKUP"::TEXT AS "ONLINE BACKUP",COL1:"TOTAL CHARGES"::REAL AS "TOTAL CHARGES",COL1:"CLTV"::NUMBER(38, 0) AS "CLTV",COL1:"CHURN VALUE"::REAL AS "CHURN VALUE",COL1:"CHURN REASON"::TEXT AS "CHURN REASON",COL1:"LATITUDE"::TEXT AS "LATITUDE",COL1:"ONLINE SECURITY"::TEXT AS "ONLINE SECURITY",COL1:"TECH SUPPORT"::TEXT AS "TECH SUPPORT",COL1:"CONTRACT"::TEXT AS "CONTRACT",COL1:"CHURN SCORE"::NUMBER(38, 0) AS "CHURN SCORE",COL1:"DEVICE PROTECTION"::TEXT AS "DEVICE PROTECTION",COL1:"ZIP CODE"::TEXT AS "ZIP CODE",COL1:"STREAMING TV"::TEXT AS "STREAMING TV",COL1:"CHURN LABEL"::BOOLEAN AS "CHURN LABEL"from "DEV"."LANDING"."RAW_PARQUET_DATA";').collect()


[Row(status='View RAW_PARQUET_DATA_VW successfully created.')]

In [27]:
dfR = session.table(view)

In [28]:
dfDemographics = dfR.select(col("CUSTOMERID"),
                             col("COUNT").alias("COUNT"),
                             translate(col("GENDER"),lit("NULL"),lit("Male")).alias("GENDER"),
                             col("SENIOR CITIZEN").alias("SENIORCITIZEN"),
                             col("PARTNER"),
                             col("DEPENDENTS")          
                            )


dfDemographics.write.mode('overwrite').saveAsTable('DEMOGRAPHICS')
dfDemographics.show()

----------------------------------------------------------------------------------
|"CUSTOMERID"  |"COUNT"  |"GENDER"  |"SENIORCITIZEN"  |"PARTNER"  |"DEPENDENTS"  |
----------------------------------------------------------------------------------
|7090-ZyCMx    |1        |Female    |False            |False      |True          |
|1364-wJXMS    |1        |Female    |False            |False      |True          |
|6564-sLgIC    |1        |Male      |True             |False      |True          |
|7853-2xheR    |1        |Male      |False            |False      |True          |
|8457-E9FuW    |1        |Female    |False            |False      |True          |
|5718-ykxBT    |1        |Male      |False            |False      |True          |
|7092-gCJX5    |1        |Male      |False            |False      |False         |
|8249-GOs7s    |1        |Male      |True             |False      |False         |
|9445-kPPEc    |1        |Male      |False            |False      |False         |
|158

In [29]:
dfLocation = dfR.select(col("CUSTOMERID"),
                         col("COUNTRY").name("COUNTRY"),
                         col("STATE").name("STATE"),
                         col("CITY").name("CITY"),
                         translate(col("ZIP CODE"),lit("NULL"),lit(0)).name("ZIPCODE"),
                         col("LAT LONG").name("LATLONG"),
                         col("LATITUDE").name("LATITUDE"),
                         col("LONGITUDE").name("LONGITUDE")       
                        )

dfLocation.write.mode('overwrite').saveAsTable('LOCATION')
dfLocation.show()

-------------------------------------------------------------------------------------------------------------------------------
|"CUSTOMERID"  |"COUNTRY"      |"STATE"     |"CITY"           |"ZIPCODE"  |"LATLONG"               |"LATITUDE"  |"LONGITUDE"  |
-------------------------------------------------------------------------------------------------------------------------------
|7090-ZyCMx    |United States  |California  |Los Angeles      |90005      |34.059281, -118.30742   |34.059281   |-118.307420  |
|1364-wJXMS    |United States  |California  |Los Angeles      |90006      |34.048013, -118.293953  |34.048013   |-118.293953  |
|6564-sLgIC    |United States  |California  |Los Angeles      |90065      |34.108833, -118.229715  |34.108833   |-118.229715  |
|7853-2xheR    |United States  |California  |La Habra         |90631      |33.940619, -117.9513    |33.940619   |-117.951300  |
|8457-E9FuW    |United States  |California  |Glendale         |91206      |34.162515, -118.203869  |34.1

In [30]:
dfServices = dfR.select(col("CUSTOMERID"),
                       col("TENURE MONTHS").name("TENUREMONTHS"),
                       iff(is_null(col("PHONE SERVICE")),lit('N'),col("PHONE SERVICE")).name("PHONESERVICE"),
                       iff(is_null(col("MULTIPLE LINES")),lit("No"),col("MULTIPLE LINES")).name("MULTIPLELINES"),
                       iff(is_null(col("INTERNET SERVICE")),lit("No"),col("INTERNET SERVICE")).name("INTERNETSERVICE"),
                       iff(is_null(col("ONLINE SECURITY")),lit("No"),col("ONLINE SECURITY")).name("ONLINESECURITY"),
                       iff(is_null(col("ONLINE BACKUP")),lit("No"),col("ONLINE BACKUP")).name("ONLINEBACKUP"),
                       iff(is_null(col("DEVICE PROTECTION")),lit("No"),col("DEVICE PROTECTION")).name("DEVICEPROTECTION"),
                       iff(is_null(col("TECH SUPPORT")),lit('N'),col("TECH SUPPORT")).name("TECHSUPPORT"),
                       iff(is_null(col("STREAMING TV")),lit("No"),col("STREAMING TV")).name("STREAMINGTV"),
                       iff(is_null(col("STREAMING MOVIES")),lit("No"),col("STREAMING MOVIES")).name("STREAMINGMOVIES"),
                       iff(is_null(col("CONTRACT")),lit("Month-to-month"),col("CONTRACT")).name("CONTRACT"),
                       iff(is_null(col("PAPERLESS BILLING")),lit('Y'),col("PAPERLESS BILLING")).name("PAPERLESSBILLING"),
                       iff(is_null(col("PAYMENT METHOD")),lit("Mailed check"),col("PAYMENT METHOD")).name("PAYMENTMETHOD"),
                       col("MONTHLY CHARGES").name("MONTHLYCHARGES"),
                       col("TOTAL CHARGES").name("TOTALCHARGES"),
                       col("CHURN VALUE").name("CHURNVALUE")        

                      )

dfServices.write.mode('overwrite').saveAsTable('SERVICES')
dfServices.show()

----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"CUSTOMERID"  |"TENUREMONTHS"  |"PHONESERVICE"  |"MULTIPLELINES"  |"INTERNETSERVICE"  |"ONLINESECURITY"     |"ONLINEBACKUP"       |"DEVICEPROTECTION"   |"TECHSUPPORT"        |"STREAMINGTV"        |"STREAMINGMOVIES"    |"CONTRACT"      |"PAPERLESSBILLING"  |"PAYMENTMETHOD"   |"MONTHLYCHARGES"  |"TOTALCHARGES"  |"CHURNVALUE"  |
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|7090-ZyCMx  

In [31]:
dfStatus = dfR.select(col("CUSTOMERID"),
                    iff(is_null(col("CHURN LABEL")),lit('N'),col("CHURN LABEL")).name("CHURNLABEL"),
                    col("CHURN VALUE").name("CHURNVALUE"),
                    col("CHURN SCORE").name("CHURNSCORE"),
                    col("CLTV").name("CLTV"),
                    iff(is_null(col("CHURN REASON")),lit("do not know"),col("CHURN REASON")).name("CHURNREASON")          
                    )

dfStatus.write.mode('overwrite').saveAsTable('STATUS')
dfStatus.show()

-----------------------------------------------------------------------------------------------------------------
|"CUSTOMERID"  |"CHURNLABEL"  |"CHURNVALUE"  |"CHURNSCORE"  |"CLTV"  |"CHURNREASON"                             |
-----------------------------------------------------------------------------------------------------------------
|7090-ZyCMx    |true          |1.0           |1             |2701    |Moved                                     |
|1364-wJXMS    |true          |1.0           |1             |5372    |Moved                                     |
|6564-sLgIC    |true          |1.0           |1             |3179    |Competitor made better offer              |
|7853-2xheR    |true          |1.0           |1             |4415    |Product dissatisfaction                   |
|8457-E9FuW    |true          |1.0           |1             |5142    |Price too high                            |
|5718-ykxBT    |true          |1.0           |1             |2484    |Poor expertise of 

In [32]:
# Lets run a query for quick sanity check
# This Query will show us the total revenue by city and contract term

dfLoc = session.table("LOCATION")
dfServ = session.table("SERVICES")

dfJoin = dfLoc.join(dfServ,dfLoc.col("CUSTOMERID") == dfServ.col("CUSTOMERID"))

dfResult = dfJoin.select(col("CITY"),
                         col("CONTRACT"),
                         col("TOTALCHARGES")).groupBy(col("CITY"),col("CONTRACT")).sum(col("TOTALCHARGES"))

dfResult.show()

----------------------------------------------------------
|"CITY"           |"CONTRACT"      |"SUM(TOTALCHARGES)"  |
----------------------------------------------------------
|Los Angeles      |Month-to-month  |3931004.7            |
|La Habra         |Month-to-month  |6828.35              |
|Glendale         |Month-to-month  |460483.05            |
|Burbank          |Month-to-month  |378354.4             |
|Ontario          |Two year        |57487.6              |
|Alpine           |Month-to-month  |69186.04999999999    |
|Borrego Springs  |Month-to-month  |94737.0              |
|Oceanside        |Month-to-month  |49559.5              |
|Niland           |Month-to-month  |24946.0              |
|San Bernardino   |Month-to-month  |253583.3             |
----------------------------------------------------------



In [33]:
##Let's create a view for data science team to begin data analysis
##To do so, join up the DEMOGRAPHICS and SERVICES tables based on CUSTOMERID

dfD = session.table('DEMOGRAPHICS')
dfS = session.table('SERVICES')
dfJ = dfD.join(dfS, using_columns='CUSTOMERID', join_type = 'left')
dfJ.select(col('GENDER'),
              col('SENIORCITIZEN'),
              col('PARTNER'),
              col('DEPENDENTS'),
              col('MULTIPLELINES'),
              col('INTERNETSERVICE'),
              col('ONLINESECURITY'),
              col('ONLINEBACKUP'),
              col('DEVICEPROTECTION'),
              col('TECHSUPPORT'),
              col('STREAMINGTV'),
              col('STREAMINGMOVIES'),
              col('CONTRACT'),
              col('PAPERLESSBILLING'),
              col('PAYMENTMETHOD'),
              col('TENUREMONTHS'),
              col('MONTHLYCHARGES'),
              col('TOTALCHARGES'),
              col('CHURNVALUE'))
dfJ.create_or_replace_view('TRAIN_DATASET')

[Row(status='View TRAIN_DATASET successfully created.')]

In [34]:
%%time

raw = session.table('TRAIN_DATASET').sample(n = 20)
data = raw.toPandas()

CPU times: user 12.4 ms, sys: 5.79 ms, total: 18.2 ms
Wall time: 405 ms


In [35]:
pd.pandas.set_option('display.max_columns', None)
data.head()

CUSTOMERID  COUNT  GENDER  SENIORCITIZEN  PARTNER  DEPENDENTS  \
0  6824-wqJmy      1    Male           True    False       False   
1  5566-Yeldk      1    Male          False     True        True   
2  8586-Vu8ew      1  Female          False    False       False   
3  6324-sOXoQ      1    Male          False    False       False   
4  4641-VHy5j      1  Female          False     True        True   

   TENUREMONTHS PHONESERVICE     MULTIPLELINES INTERNETSERVICE  \
0            33           No  No phone service             DSL   
1            54          Yes               Yes              No   
2             1          Yes                No     Fiber optic   
3            32          Yes                No              No   
4            72          Yes               Yes             DSL   

        ONLINESECURITY         ONLINEBACKUP     DEVICEPROTECTION  \
0                   No                   No                  Yes   
1  No internet service  No internet service  No internet service   
2                   No                   No                  Yes   
3  No internet service  No internet service  No internet service   
4                  Yes                  Yes                  Yes   

           TECHSUPPORT          STREAMINGTV      STREAMINGMOVIES  \
0                  Yes                  Yes                  Yes   
1  No internet service  No internet service  No internet service   
2                   No                   No                   No   
3  No internet service  No internet service  No internet service   
4                  Yes                   No                  Yes   

         CONTRACT PAPERLESSBILLING              PAYMENTMETHOD  MONTHLYCHARGES  \
0  Month-to-month             true           Electronic check           54.60   
1        Two year             true  Bank transfer (automatic)           24.05   
2  Month-to-month            false           Electronic check           75.30   
3        One year            false               Mailed check           19.80   
4        Two year             true  Bank transfer (automatic)           79.50   

   TOTALCHARGES  CHURNVALUE  
0        1803.7         0.0  
1        1230.9         0.0  
2          75.3         1.0  
3         607.7         0.0  
4        5661.7         0.0

In [36]:
session.close()